##Important Module

In [ ]:
!pip install nipype
!pip install nilearn
!pip install bids

In [15]:
import os
import json
import numpy as np
import pylab as plt
import nibabel as nb
from nipype import Node, Workflow
from nipype.interfaces.io import SelectFiles, DataSink
from nipype.interfaces.utility import IdentityInterface
from nilearn.plotting import plot_anat
from os.path import join as opj
from nipype.interfaces.matlab import MatlabCommand
from nipype.interfaces.spm import Normalize12, NewSegment
from nipype.algorithms.misc import Gunzip
from bids.layout import BIDSLayout
from nipype.algorithms.rapidart import ArtifactDetect
from nipype.interfaces.fsl import (BET,ExtractROI,FAST,FLIRT,MCFLIRT,ImageMaths,SliceTimer,Threshold,Smooth)


##Experiment Parameters

In [ ]:
data_dir='/home/ubuntu/Documents/BIDs/XP2'
experiment_dir='/home/ubuntu/Documents/windowshare/output/XP2'
output_dir='datasink'
working_dir='workingdir'

subject_list_2d=['xp204','xp205','xp207','xp210','xp212','xp213','xp215','xp216','xp217','xp221','xp223']
subject_list=['xp201','xp202','xp203','xp206','xp208','xp209','xp211','xp214','xp218','xp219','xp220','xp222']

task_list=['1dNF']
run_list=['01','02']
fwhm_list=[6]

#isometric voxel resolution
desired_voxel_iso = 4

##Metada Info from BIDS layout

In [ ]:
#FIND INFO FROM BIDS LAYOUT
func_file='/home/ubuntu/Documents/BIDs/XP2/sub-xp201/func/sub-xp201_task-1dNF_run-01_bold.nii.gz'
layout_data=BIDSLayout(data_dir)
sub_list=layout_data.get_subjects()
layout_data.get_tasks()
TR=layout_data.get_metadata(func_file)["RepetitionTime"]
sliceTiming=layout_data.get_metadata(func_file)["SliceTiming"]
slice_order=[sliceTiming.index(x) for x in sorted(sliceTiming)]

print("TR: "+str(TR))
print(str(slice_order))
print("image dim: "+ str(layout_data.get_metadata(func_file)["PhaseEncodingSteps"])+ " X "+ str(layout_data.get_metadata(func_file)["PhaseEncodingSteps"])+ " X " + str(len(sliceTiming)))

##Main Workflow

In [ ]:
#SLICE TIME CORRECTION
slicetimer = Node(SliceTimer(index_dir=False,
                             interleaved=True,
                             output_type='NIFTI',
                             time_repetition=TR),
                  name="slicetimer")

#MOTION CORRECTION
mcflirt=Node(MCFLIRT(mean_vol=True,
                    save_plots=True,
                     output_type='NIFTI'),
             name="mcflirt")

#ARTIFACT DETECTION
art=Node(ArtifactDetect(norm_threshold=2,
                       zintensity_threshold=2,
                       mask_type='spm_global',
                       parameter_source='FSL',
                       use_differences=[True,False],
                       plot_type='svg'), name='art')


#COREGISTRATION FUNC-ANAT IMAGEs
coreg = Node(FLIRT(dof=6,
                   cost='bbr',
                   schedule='/usr/local/fsl/etc/flirtsch/bbr.sch',
                   output_type='NIFTI'),
             name="coreg")

#SMOOTH

smooth = Node(Smooth(), name="smooth")
smooth.iterables = ("fwhm", fwhm_list)

##Coregistration workflow

In [ ]:
# BET -Skullstrip anatomical Image
bet_anat=Node(BET(frac=0.4,
                 robust=True,
                 output_type='NIFTI_GZ'),
             name="bet_anat")
#SEGMENTATION

# with FAST 
segmentation=Node(FAST(output_type='NIFTI_GZ'),name="segmentation")

#with SPM
#segmentation = Node(NewSegment(tissues=tissues), name='segmentation')


# Select WM segmentation file from segmentation output
# for fast
def get_wm(files):
    return files[-1]

#for spm
#def get_wm(files):
 #   return files[1][0]

#Threshold white matter probablity map 
threshold_WM = Node(Threshold(thresh=0.5,
                              args='-bin',
                              output_type='NIFTI'),
                name="threshold_WM")

#FLIRT - pre-alignement (6 DOF) of functional to anatomical image
coreg_pre=Node(FLIRT(dof=6, output_type='NIFTI_GZ'),name="coreg_pre")

#FLIRT - coregistration of functional to anatomical with BBR
coreg_bbr=Node(FLIRT(dof=6,cost='bbr',
                    schedule=opj(os.getenv('FSLDIR'),'etc/flirtsch/bbr.sch'),
                    output_type='NIFTI_GZ'), name='coreg_bbr')

#APPLY coregistration warp to functional image
applywarp = Node(FLIRT(interp='spline',
                       apply_isoxfm=desired_voxel_iso,
                       output_type='NIFTI'),
                 name="applywarp")

#apply coregistration warp to mean file
applywarp_mean=Node(FLIRT(interp='spline',
                         apply_isoxfm=desired_voxel_iso,
                         output_type='NIFTI_GZ'),
                 name="applywarp_mean")

##create a coregistration workflow and connect nodes

In [ ]:
coregwf=Workflow(name='coregwf')
coregwf.base_dir=opj(experiment_dir,working_dir)


coregwf.connect([
              #(bet_anat,segmentation,[('out_file','channel_files')]),
               (bet_anat,segmentation,[('out_file','in_files')]),
               (segmentation, threshold_WM, [(('partial_volume_files', get_wm),
                                          'in_file')]),
               #  (segmentation, threshold_WM, [(('native_class_images', get_wm),
               #                          'in_file')]),
                (bet_anat,coreg_pre,[('out_file','reference')]),
                (threshold_WM,coreg_bbr,[('out_file','wm_seg')]),
                (coreg_pre,coreg_bbr,[('out_matrix_file','in_matrix_file')]),
                (coreg_bbr,applywarp,[('out_matrix_file','in_matrix_file')]),
                (bet_anat,applywarp,[('out_file','reference')]),
                (coreg_bbr,applywarp_mean,[('out_matrix_file','in_matrix_file')]),
                (bet_anat,applywarp_mean,[('out_file','reference')]),
                ])

##data input and output

In [ ]:
#Inforsource- to iterate over list of subject names
# zrun_id because I want it to be at the end of the folder tree (apparently IdentityInterface follows alphabetic order)
infosource=Node(IdentityInterface(fields=['subject_id','task_id','zrun_id']), 
               name="infosource")
infosource.iterables=[('subject_id',subject_list),
                     ('task_id',task_list),
                     ('zrun_id',run_list)]

# String template with {}-based strings
templates = {'anat': 'sub-{subject_id}/anat/'
                     'sub-{subject_id}_T1w.nii.gz',
             'func': 'sub-{subject_id}/func/'
                     'sub-{subject_id}_task-{task_id}_run-{zrun_id}_bold.nii.gz'}

# Create SelectFiles node
selectfiles = Node(SelectFiles(templates,
                      base_directory=data_dir,
                      sort_filelist=True),
          name='selectfiles')


# DataSink- creates output folder for important outputs
datasink=Node(DataSink(base_directory=experiment_dir,
                      container=output_dir), name="datasink")

substitutions=[('_task_id_','/task-'),
               ('_subject_id_','sub-'),
              ('_zrun_id_','/run-'),
              ('_fwhm_','fwhm-'),
              ('_roi',''),
              ('_mcf',''),
              ('_st',''),
              ('_flirt',''),
              ('_smooth',''),
              ('.nii_mean_reg','_mean'),
              ('.nii.par','.par')]

subjFolders=[('fwhm-%s/' % f, 'fwhm-%s-' % f) for f in fwhm_list]
substitutions.extend(subjFolders)
datasink.inputs.substitutions=substitutions

##Main Workflow

In [ ]:
preproc=Workflow(name='preprocessing',base_dir=opj(experiment_dir,working_dir))

preproc.connect([(infosource,selectfiles,[('subject_id','subject_id'),
                                          ('task_id','task_id'),
                                          ('zrun_id','zrun_id')]),
                (selectfiles,mcflirt,[('func','in_file')]),
                (mcflirt,slicetimer,[('out_file','in_file')]),
                (selectfiles,coregwf,[('anat','bet_anat.in_file'),
                                     ('anat','coreg_bbr.reference')]),
                (mcflirt,coregwf,[('mean_img','coreg_pre.in_file'),
                                 ('mean_img','coreg_bbr.in_file'),
                                 ('mean_img','applywarp_mean.in_file')]),
                (slicetimer,coregwf,[('slice_time_corrected_file','applywarp.in_file')]),
                (coregwf,smooth,[('applywarp.out_file','in_file')]),
               # (coregwf,normalize,[('applywarp.out_file','image_to_align')]),
               #(normalize,smooth,[('normalized_image','in_file')]),
                 
                (mcflirt,datasink,[('par_file','preproc.@par')]),
                (smooth,datasink,[('smoothed_file','preproc.@smooth')]),
               # (normalize,datasink,[('normalized_files','norm_spm.@files'),
                        #            ('normalized_image','norm_spm.@image')]),
                (coregwf,datasink,[('applywarp_mean.out_file','preproc.@mean')]),
                 
                (coregwf,art,[('applywarp.out_file','realigned_files')]),
                (mcflirt,art,[('par_file','realignment_parameters')]),
                 
                (coregwf,datasink,[('coreg_bbr.out_matrix_file','preproc.@mat_file'),
                                  ('bet_anat.out_file','preproc.@brain')]),
                (art,datasink,[('outlier_files','preproc.@outlier_files'),
                              ('plot_files','preproc.@plot_files')]),
                ])


#Visualize

In [ ]:
# Create preproc output graph
preproc.write_graph(graph2use='colored', format='png', simple_form=True)

# Visualize the graph
from IPython.display import Image
Image(filename=opj(preproc.base_dir,'preprocessing','graph.png'), width=750)

#Run

In [ ]:
#preproc.run('MultiProc', plugin_args={'n_procs': 4})
preproc.run()

##fMRI 1st level analysis
Individual (1st level) analysis -adapted from nipype tutorial (Miykael Notters and Gi Lio examples)

In this analysis workflow we will:
* Extract stimuli time from json files (BIDs)
* Specify the 1st level GLM (General Linear Model)
* Specify Contrasts to compute
* Estimate contrasts (in this example: con_0001:NF vs REST [0 1], con_0002 NF>REST [-1 1]
* Normalize contrasts


##Imports

In [4]:
from os.path import join as opj
import json
from nipype.interfaces.spm import Level1Design, EstimateModel, EstimateContrast, Normalize12
tpm_img ='/home/ubuntu/Documents/MATLAB/spm12/tpm/TPM.nii' # normalization template

from nipype.algorithms.modelgen import SpecifySPMModel
from nipype.interfaces.utility import Function, IdentityInterface
from nipype.interfaces.io import SelectFiles, DataSink
from nipype import Workflow, Node
from bids.layout import BIDSLayout
from nipype.algorithms.misc import Gunzip

import numpy as np
from matplotlib import pyplot as plt
from scipy.io import loadmat